# 0. Libraries

In [1]:
import cv2 # opencv
import mediapipe as mp
import numpy as np
mp_drawing = mp.solutions.drawing_utils # drawing utilities
mp_pose = mp.solutions.pose # pose estimation model

In [2]:
# Video feed
cap = cv2.VideoCapture(0) # This allow to use any camera connected. Number represents the camera
while cap.isOpened():
    ret, frame = cap.read() # frame is for each image captured
    cv2.imshow("Mediapipe Feed", frame) # Show the image captured

    if cv2.waitKey(10) & 0xFF == ord("q"): # Press q to close video
        break

cap.release()
cv2.destroyAllWindows()

# 1. Make Detections

In [ ]:
# Video feed
cap = cv2.VideoCapture(0) # This allow to use any camera connected. Number represents the camera

# Setup meediapipe instance
# Accuracy level. Higher score higher confidence required for detection
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:

    while cap.isOpened():
        ret, frame = cap.read() # frame is for each image captured

        # Recolor image to RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False # To improve performance

        # Make detection
        results = pose.process(image)

        # Recolor back to BGR
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        # Render detections
        # results.pose_landmarks shows the position each individual point of the body
        # mp_pose.POSE_CONNECTIONS shows the connections between each individual point of the body
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                  mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2),
                                  mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                                  )

        #print(results)


        cv2.imshow("Mediapipe Feed", image) # Show the image captured

        if cv2.waitKey(10) & 0xFF == ord("q"): # Press q to close video
            break

    cap.release()
    cv2.destroyAllWindows()

# 2. Determining Joints

<img src="https://i.imgur.com/3j8BPdc.png" style="height:350px">

In [9]:
# Video feed
cap = cv2.VideoCapture(0) # This allow to use any camera connected. Number represents the camera

# Setup meediapipe instance
# Accuracy level. Higher score higher confidence required for detection
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:

    while cap.isOpened():
        ret, frame = cap.read() # frame is for each image captured

        # Recolor image to RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False # To improve performance

        # Make detection
        results = pose.process(image)

        # Recolor back to BGR
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        # Extract landmarks
        try: # try because sometimes it could not be detected
            landmarks = results.pose_landmarks.landmark
            #print(landmarks)
        except:
            pass

        # Render detections
        # results.pose_landmarks shows the position each individual point of the body
        # mp_pose.POSE_CONNECTIONS shows the connections between each individual point of the body
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                  mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2),
                                  mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                                  )

        #print(results)


        cv2.imshow("Mediapipe Feed", image) # Show the image captured

        if cv2.waitKey(10) & 0xFF == ord("q"): # Press q to close video
            break

    cap.release()
    cv2.destroyAllWindows()

In [ ]:
for landmark in mp_pose.PoseLandmark:
    print(landmark.name, landmark.value)

33

In [ ]:
mp_pose.PoseLandmark.LEFT_SHOULDER

In [10]:
landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value]


x: 0.179282799
y: 0.525727689
z: -1.49277699
visibility: 0.997917891

In [31]:
landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value]

x: 1.168347
y: 1.45901144
z: -0.679086089
visibility: 0.0602062382

In [32]:
landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value]

x: 1.08796525
y: 1.92019498
z: -1.19966304
visibility: 0.011106573

# 3. Calculate Angles for bicep curl

In [11]:
# This function calculates angle between three points of any body
def calculate_angle(a,b,c):

    # remember that each point has X,Y,Z values ([0],[1],[2])
    a = np.array(a) # First (shoulder in this case)
    b = np.array(b) # Mid (elbow in this case)
    c = np.array(c) # End (wrist in this case)

    radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
    angle = np.abs(radians*180.0/np.pi)

    if angle > 180.0: # Angle between 0 and 180 (max angle of an arm is 180 actually)
        angle = 360-angle

    return angle

In [12]:
shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,
            landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
elbow = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x,
         landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
wrist = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x,
         landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]

calculate_angle(shoulder, elbow, wrist)

156.69158877974522

In [16]:
# Video feed
cap = cv2.VideoCapture(0) # This allow to use any camera connected. Number represents the camera

# Setup meediapipe instance
# Accuracy level. Higher score higher confidence required for detection
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:

    while cap.isOpened():
        ret, frame = cap.read() # frame is for each image captured

        # Recolor image to RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False # To improve performance

        # Make detection
        results = pose.process(image)

        # Recolor back to BGR
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        # Extract landmarks
        try: # try because sometimes it could not be detected
            landmarks = results.pose_landmarks.landmark

            # Get coordinates
            shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,
                        landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
            elbow = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x,
                     landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
            wrist = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x,
                     landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]
            
            # Calculate angle
            angle = calculate_angle(shoulder, elbow, wrist)
            angle = (angle).round(2)

            # Visualize angle
            cv2.putText(image, str(angle),
                        # Set position of angle text
                        # 640 and 480 are the width and height of the camera image
                        tuple(np.multiply(elbow, [640, 480]).astype(int)),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA # Text and line style
                        )


        except:
            pass

        # Render detections
        # results.pose_landmarks shows the position each individual point of the body
        # mp_pose.POSE_CONNECTIONS shows the connections between each individual point of the body
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                  mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2),
                                  mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                                  )

        #print(results)


        cv2.imshow("Mediapipe Feed", image) # Show the image captured

        if cv2.waitKey(10) & 0xFF == ord("q"): # Press q to close video
            break

    cap.release()
    cv2.destroyAllWindows()

# 4. Curl Counter

In [ ]:
# Video feed
cap = cv2.VideoCapture(0) # This allow to use any camera connected. Number represents the camera

# Curl counter variables
counter = 0
stage = None # "up" or "down" part of the curl


# Setup meediapipe instance
# Accuracy level. Higher score higher confidence required for detection
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:

    while cap.isOpened():
        ret, frame = cap.read() # frame is for each image captured

        # Recolor image to RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False # To improve performance

        # Make detection
        results = pose.process(image)

        # Recolor back to BGR
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        # Extract landmarks
        try: # try because sometimes it could not be detected
            landmarks = results.pose_landmarks.landmark

            # Get coordinates
            shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,
                        landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
            elbow = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x,
                     landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
            wrist = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x,
                     landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]
            
            # Calculate angle
            angle = calculate_angle(shoulder, elbow, wrist)
            angle = (angle).round(2)

            # Visualize angle
            cv2.putText(image, str(angle),
                        # Set position of angle text
                        # 640 and 480 are the width and height of the camera image
                        tuple(np.multiply(elbow, [640, 480]).astype(int)),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA # Text and line style
                        )
            
            # Curl counter logic
            if angle > 160:
                stage = "down"
            if angle < 30 and stage == "down":
                stage = "up"
                counter += 1
                print(counter)

        except:
            pass

        # Render curl counter

        # Setup status box
        cv2.rectangle(image, (0,0), (225,73), (245,117,16), -1)

        # Rep data
        cv2.putText(image, 'REPS', (15,12),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
        cv2.putText(image, str(counter),
                    (10,60), cv2.FONT_HERSHEY_SIMPLEX, 2, (255,255,255), 2, cv2.LINE_AA)
        
        # Stage data
        cv2.putText(image, 'STAGE', (65,12),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
        cv2.putText(image, str(stage),
                    (60,60), cv2.FONT_HERSHEY_SIMPLEX, 2, (255,255,255), 2, cv2.LINE_AA)
        


        # Render detections
        # results.pose_landmarks shows the position each individual point of the body
        # mp_pose.POSE_CONNECTIONS shows the connections between each individual point of the body
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                  mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2),
                                  mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                                  )

        #print(results)


        cv2.imshow("Mediapipe Feed", image) # Show the image captured

        if cv2.waitKey(10) & 0xFF == ord("q"): # Press q to close video
            break

    cap.release()
    cv2.destroyAllWindows()

# Siguiente paso

Probar lo anterior en Streamlit